In [4]:
# 参考 https://qiita.com/God_KonaBanana/items/2cf829172087d2423f58

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger

In [5]:
# 環境の指定
n_categories=5 #分類数
batch_size=32
data_dir='../scraping/data'
file_name='vgg16_kankitu'



In [9]:
# モデル構築 時間かかる

# VGG16モデル
base_model = VGG16(weights="imagenet", include_top=False,
                   input_tensor=Input(shape=(224, 224, 3)))

# 追加のレイヤー
vgg16_output = base_model.output
GAP2D = GlobalAveragePooling2D()(vgg16_output)
Dense1 = Dense(1024, activation="relu")(GAP2D)
prediction = Dense(n_categories, activation="softmax")(Dense1)

# モデルを新たに構築
model = Model(inputs=base_model.input, outputs=prediction)

In [10]:
# 重さの指定
for layer in base_model.layers[:15]:
    layer.trainable = False

In [11]:
# compile
model.compile(optimizer=SGD(),
              loss = "categorical_crossentropy",
              metrics = ["accuracy"])




In [12]:
# summary
model.summary()



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
# データの準備
# train, test
train_path = data_dir + "/train"
test_path = data_dir + "/test"


In [ ]:
# data generator

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1.0/255)
